In [2]:
from tensorflow.keras.models import load_model

# Load the .keras model
model = load_model("/content/CFP_vgg19_model.keras")
# Save the model in .h5 format
model.save("/content/CFP_vgg19_model222.h5")


In [7]:
!pip install h5py
import h5py

try:
  with h5py.File('/content/OCT_vgg19_model.h5', 'r') as f:
    print(f.keys())
except OSError as e:
  print(f"Error opening file: {e}")

<KeysViewHDF5 ['model_weights', 'optimizer_weights']>


In [9]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.applications.vgg19 import preprocess_input
from tensorflow.keras.preprocessing import image


classification_model = tf.keras.models.load_model('/content/oct_cfp_vgg19_model.h5')


cfp_model = tf.keras.models.load_model('/content/CFP_vgg19_model222.h5')
oct_model = tf.keras.models.load_model('/content/OCT_vgg19_model.h5')

# Class names for the classification model
class_names = ["CFP", "OCT"]


cfp_class_names = ['Normal', 'cataract', 'diabetic_retinopathy', 'glaucoma']
oct_class_names = ['CNV', 'DME', 'DRUSEN', 'NORMAL']

def classify_image(image_path):
    """Classify the image as CFP or OCT."""
    img = image.load_img(image_path, target_size=(224, 224))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array = preprocess_input(img_array)

    predictions = classification_model.predict(img_array)
    predicted_class = np.argmax(predictions, axis=1)[0]
    class_name = class_names[predicted_class]

    return class_name, img_array

def predict_disease(image_array, model, class_names):
    """Use the specific model (CFP or OCT) to predict diseases."""
    predictions = model.predict(image_array)
    percentages = predictions[0] * 100
    predicted_class = np.argmax(predictions)


    print("\nPercentages per class:")
    for i, class_name in enumerate(class_names):
        print(f"  {class_name}: {percentages[i]:.2f}%")


    return class_names[predicted_class]


image_path = input("Enter the path of the image: ")

# Classify the image as CFP or OCT
predicted_class, img_array = classify_image(image_path)
print(f"Classification model predicts this image as: {predicted_class}")

# Choose the appropriate model based on the classification
if predicted_class == "CFP":
    disease_model = cfp_model
    disease_class_names = cfp_class_names
else:
    disease_model = oct_model
    disease_class_names = oct_class_names

# Predict disease using the appropriate model
predicted_disease = predict_disease(img_array, disease_model, disease_class_names)
print(f"\nPredicted Class: {predicted_disease}")



Enter the path of the image: /content/diabetic_retinopathy (97).jpg
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
Classification model predicts this image as: CFP
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 935ms/step

Percentages per class:
  Normal: 0.00%
  cataract: 0.00%
  diabetic_retinopathy: 100.00%
  glaucoma: 0.00%

Predicted Class: diabetic_retinopathy
